In [3]:
import sqlparse                          # SQL Parsing을 위한 library
from sql_metadata import Parser          # SQL Metadata 추출을 위한 library

sql_text = """
                SELECT std.task_id ,
                COUNT( 1 )
                FROM   (
                        SELECT std.task_id
                        FROM   some_task_detail std
                        WHERE  std.status = 1
                ) a join (
                                SELECT st.task_id
                                FROM   some_task st
                                WHERE  st.task_type_id = 80
                ) b ON a.task_id = b.task_id
                GROUP  BY std.task_id
                ORDER  BY std.task_id
"""

sql_text = sqlparse.format(sql_text, strip_comments=True) # 주석 제거 (sql_metadata에서 주석제거는 버그가 있음.)
sql_tables = Parser(sql_text).tables
sql_columns = Parser(sql_text).columns
table_alias = Parser(sql_text).tables_aliases

sql_text = sql_text.replace("\"","\'") # SQL에 " 따옴표가 있는 경우, 컬럼명으로 인식 (" -> ' 변경 필요)
sql_text = Parser(sql_text).columns_dict

print("--------------------------------------------------------------------------")
print("[테이블 목록]   : {0}\n".format(sql_tables))
print("Where절 목록    : {0}".format(sql_text.get("where")))    # 테이블 조인이 있는 경우는 "테이블명.컬럼명", 단독 테이블인 경우는 "컬럼명"
print("조인 컬럼 목록  : {0}".format(sql_text.get("join")))    # "테이블명.컬럼명"
print("Group by절 목록 : {0}".format(sql_text.get("group_by"))) # 테이블 조인이 있는 경우는 "테이블명.컬럼명", 단독 테이블인 경우는 "컬럼명"
print("Order by절 목록 : {0}".format(sql_text.get("order_by"))) # 테이블 조인이 있는 경우는 "테이블명.컬럼명", 단독 테이블인 경우는 "컬럼명"
print("--------------------------------------------------------------------------\n")

if "where" not in sql_text and "join" not in sql_text and "order_by" not in sql_text:

        print("[분석 결과] : where절과 테이블 조인, order by절이 없는 SQL로 분석 대상이 아닙니다.")

elif "where" not in sql_text and "join" not in sql_text and "order_by" in sql_text:

        # 1개 테이블에서 데이터 조회하는 쿼리가 Order By가 없고, order by 후 rownum 처리한다면 인덱스 생성 필요.
        print("[분석 결과] : where절과 테이블 조인이 없고, order by절만 있는 SQL입니다.")

elif "where" not in sql_text and "join" in sql_text and "order_by" in sql_text:

        print("[분석 결과] : where절은 없고, 테이블 조인과 order by절만 있는 SQL입니다.")

elif "where" in sql_text and "join" not in sql_text:

        # 테이블 조인이 없는 1개 테이블에서 조회하는 쿼리가 Full Table Scan이라면 반드시 분석대상 (FTS 튜닝 대상 선별 시 활용)
        print("[분석 결과] : where절과 테이블 조인이 없는 SQL입니다. 인덱스 점검 대상 SQL입니다.")

elif "where" in sql_text and "join" in sql_text:

        where_col = sql_text["where"]
        join_col = sql_text["join"]            # 조인 컬럼은 NL Join일 경우, 반드시 인덱스 있어야 함.

        print("[분석 결과] : where절과 테이블 조인이 있는 SQL입니다. 인덱스 점검 대상 SQL입니다.")
        print("{0} 테이블의 Where절에 사용되는 컬럼은 {1} 입니다.".format(sql_tables, where_col))
        print("{0} 테이블의 조인에 사용되는 컬럼은 {1} 입니다.".format(sql_tables, join_col))
else:
        print("아직 분석 조건을 설정하지 않았습니다.")

--------------------------------------------------------------------------
[테이블 목록]   : ['some_task_detail', 'some_task']

Where절 목록    : ['some_task_detail.status', 'some_task.task_type_id']
조인 컬럼 목록  : ['some_task_detail.task_id', 'some_task.task_id']
Group by절 목록 : ['some_task_detail.task_id']
Order by절 목록 : ['some_task_detail.task_id']
--------------------------------------------------------------------------

[분석 결과] : where절과 테이블 조인이 있는 SQL입니다. 인덱스 점검 대상 SQL입니다.
['some_task_detail', 'some_task'] 테이블의 Where절에 사용되는 컬럼은 ['some_task_detail.status', 'some_task.task_type_id'] 입니다.
['some_task_detail', 'some_task'] 테이블의 조인에 사용되는 컬럼은 ['some_task_detail.task_id', 'some_task.task_id'] 입니다.


In [2]:
!pip install sql_metadata